In this notebook, we shall create a multi-layer perceptron (MLP) that performs the XOR logical operation.

If running this notebook in Colab, please ensure that your Hugging Face `HF_TOKEN` is added to your Colab secrets.

Alternatively, please login to Hugging Face by running the following cell.

In [1]:
# !huggingface-cli login

Let's create two column vectors containing `0`s and `1`s.

In [2]:
import torch

batch = {'a': torch.tensor([[0.], [0.], [1.], [1.]]), 'b': torch.tensor([[0.], [1.], [0.], [1.]])}
batch

{'a': tensor([[0.],
         [0.],
         [1.],
         [1.]]),
 'b': tensor([[0.],
         [1.],
         [0.],
         [1.]])}

Our objective is to generate the following truth table using an MLP.

| A | B | C |
| - | - | - |
| 0 | 0 | 0 |
| 0 | 1 | 1 |
| 1 | 0 | 1 |
| 1 | 1 | 0 |

In [3]:
import torch.nn as nn
from huggingface_hub import PyTorchModelHubMixin

class XOR(nn.Module, PyTorchModelHubMixin):
    """
    A multi-layer perceptron that performs the XOR logical computation.
    It takes as input two column vectors of zeros and ones. It outputs a single column vector of zeros and ones.
    """

    def __init__(self):
        super().__init__()
        self.layer0_weight = torch.tensor([[1., 1.], [1., 1.]])
        self.layer0_bias = torch.tensor([-1.5, -0.5])
        self.layer1_weight = torch.tensor([[-1.], [1.]])
        self.layer1_bias = torch.tensor([-0.5])

    def heaviside(self, x):
        return (x >= 0).float()

    def forward(self, x):
        inputs = torch.cat([x['a'], x['b']], dim=1)
        out = self.heaviside(inputs @ self.layer0_weight + self.layer0_bias)
        out = self.heaviside(out @ self.layer1_weight + self.layer1_bias)
        return out

In [4]:
model = XOR()
model

XOR()

Forward pass:

In [5]:
model(batch)

tensor([[0.],
        [1.],
        [1.],
        [0.]])

The XOR operation works.

Let's push our model to the Hugging Face Hub.

In [6]:
# model.push_to_hub("xor", commit_message="pushing XOR")